<a href="https://colab.research.google.com/github/matjesg/DeepFLaSH/blob/master/tune_and_predict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook is optmizied to be executed on Google Colab (https://colab.research.google.com).


*   Please read the instructions carefully.
*   Press the the *play* butten to execute the cells. It will show up between \[     \] on the left side of the code cells. 
*   Run the cells consecutively.

**Note:** You can predict your images without fine-tuning the model. For this, skip the Section 2  *Train model on the new data*.

>[Configuration](#scrollTo=rPRgwBd5tmGp)

>>[Set up Google Colab Environment](#scrollTo=Zm5wDM15c_cw)

>>[Choose base model](#scrollTo=c5ICgEznc_dB)

>[Train model on the new data](#scrollTo=6e59NnbQc_dL)

>>[Provide image training data](#scrollTo=V1588m6AtbhK)

>>>[Upload your images and masks (segmentation maps)](#scrollTo=5pLJdaY1Ymmh)

>>>[Use example images](#scrollTo=uB9vyeR3bVI0)

>>>[Plot images and masks](#scrollTo=8cD0AY6uZn71)

>>[Model training](#scrollTo=fV6yuPVuuL9P)

>>>[Check results on train data](#scrollTo=hv5bXNnJc_dX)

>>>[Plot all images and joined mask](#scrollTo=5n3FfCHnc_do)

>[Create segmentation maps for new images](#scrollTo=MAVMhDs1c_dg)

>>[Compare segmentation results](#scrollTo=Tg09Afx3Mwra)

>>[Save and download predicted masks](#scrollTo=NZ7kRhs9c_ds)



# Configuration
In this section, you will set up the training environment and choose your base model.

## Set up Google Colab Environment

In [ ]:
!git clone https://github.com/matjesg/DeepFLaSH.git
import os
import sys
ROOT_DIR = os.path.abspath("DeepFLaSH")
sys.path.append(ROOT_DIR)\
    
import numpy as np
from unet import utils
from unet import colab_utils
from google.colab import files

## Choose base model

Look at the images and masks (segmentation maps) below. Which are more similar to yours?


**cFOS** image and mask \\
<img src="https://github.com/matjesg/DeepFlaSH/raw/master/sample_images/_examples/cFOS.png" width="250" height="250" alt="Parv">
<img src="https://github.com/matjesg/DeepFlaSH/raw/master/sample_images/_examples/cFOS_mask.png" width="250" height="250" alt="cFOS Mask">

**Parv** image and mask \\
<img src="https://github.com/matjesg/DeepFlaSH/raw/master/sample_images/_examples/Parv.png" width="250" height="250" alt="Parv">
<img src="https://github.com/matjesg/DeepFlaSH/raw/master/sample_images/_examples/Parv_mask.png" width="250" height="250" alt="Parv Mask">







**Which model do you want to choose:**

* pretrained: 'cFOS', 'cFOS_Inns1', 'cFOS_Inns2', 'cFOS_Mue' or 'Parv'
* untrained model (no pretrained weights): 'new'

In [ ]:
model = utils.load_unet('cFOS')

# Train model on the new data 
In this section, you can train your own model.

## Provide image training data
Either **upload your own images** or use the **transfer learning images from our repository**.

### Upload your images and masks (segmentation maps)
The training images and masks should reflect the diversity of your dataset.
* For fine-tuning a model, we recommend at least five image-mask pairs.
* For training a new model from scratch we recommend about 30 image-mask pairs.
* Make sure that both images and masks follow the same naming conventions, e.g. '01_img.tif' and '01_mask.tif'.
* Images will be resized to a 1024x1024 pixel resolution (greyscale, one channel).
* Typical filetypes are allowed (e.g., tif, png)

**Images:**

In [ ]:
img_names, img_list = colab_utils.upload_files()

**Masks**:

In [ ]:
msk_names, msk_list = colab_utils.upload_files()

### Use example images

In [ ]:
img_names, img_list = colab_utils.load_samples(path = 'transfer_learning/train', suffix = 'new')
msk_names, msk_list = colab_utils.load_samples(path = 'transfer_learning/train', suffix = 'expert')

### Plot images and masks 

Check if images and masks are correctly assigned. If not, adjust your filenames and upload the images and masks again.

In [ ]:
utils.plot_image_and_mask(img_names = img_names, img_list = img_list,
                          msk_names = msk_names, msk_list = msk_list)

## Model training

**Training duration (epochs)**

One epoch is when an entire (augemented) dataset is passed through the neural network for training. 
* We recommend about 50 epochs for fine-tuning and at least 100 epochs for a new model. 
* Choose a higher number if your images are very dissimilar to the sample images above.

In [ ]:
epochs = 50

**Data Augmentation**
Create data augmentation generator for images and masks

In [ ]:
train_generator = utils.create_generator(img_list, msk_list)

**Train model**

In [ ]:
model.fit_generator(train_generator,
                    steps_per_epoch=int(np.ceil(len(img_list)/4.)),
                    epochs=epochs)

### Check results on train data

Predict masks and calculate Jaccard Similarity

In [ ]:
# Predict masks with the U-net
pred_train = model.predict(np.asarray(img_list))
msk_train_list = [pred_train[i] for i in range(pred_train.shape[0])]

# Calculate Jaccard Similarity
jac_train = utils.jaccard_sim(msk_list, msk_train_list)
jac_train

### Plot all images and joined mask

The joined mask consists of the manual/expert segmentation mask and U-net prediction.

Color code: 
- white = merge
- magenta = U-net only
- green = original/expert only

In [ ]:
join_list = [utils.join_masks(msk_train_list[i], msk_list[i]) for i in range(len(msk_list))]
utils.plot_image_and_mask(img_names = img_names, img_list = img_list,
                          msk_names = jac_train, msk_list = join_list,
                          msk_head = 'Jaccard Similarity')
                    

# Create segmentation maps for new images
In this section, you can upload unlabelled images and predict the segmentation map (mask).

**Upload images**

*   Images will be resized to a 1024x1024 pixel resolution (greyscale, one channel).
*   Typical filetypes are allowed (e.g., tif, png)

In [ ]:
img_new_names, img_new_list = colab_utils.upload_files()

**Predict masks (segmentation maps) with the U-net**

In [ ]:
pred_new = model.predict(np.asarray(img_new_list))
pred_new_list = [pred_new[i] for i in range(pred_new.shape[0])]

**Plot results**
Look at the segmentation results of the U-net.

In [ ]:
utils.plot_image_and_mask(img_names = img_new_names, img_list = img_new_list,
                    msk_names = img_new_names, msk_list = pred_new_list)

## Compare segmentation results
If you already have segmentation maps of the above images at your disposal, you can upload them here for comparison.

**Upload new segmentation maps (masks)**

* Make sure that both images and masks follow the same naming conventions, e.g. '01_img.tif' and '01_mask.tif'.
* Images will be resized to a 1024x1024 pixel resolution (greyscale, one channel).

In [ ]:
msk_new_names, msk_new_list = colab_utils.upload_files()

**Plot comparison**

Color code: 
- white = merge
- magenta = U-net only
- green = original/expert only

In [ ]:
# Calculate Jaccard Similarity
jac_test = utils.jaccard_sim(pred_new_list, msk_new_list)

join_new_list = [utils.join_masks(pred_new_list[i], msk_new_list[i]) for i in range(len(msk_list))]
utils.plot_image_and_mask(img_names = img_new_names, img_list = img_new_list,
                          msk_names = jac_test, msk_list = pred_new_list,
                          msk_name = 'Jaccard Similarity')

## Save and download predicted masks

In [ ]:
utils.saveMasks(pred_new_list, img_new_names)
!zip -r masks.zip masks
files.download('masks.zip')